In [27]:
import os
print(os.getcwd())

c:\Users\User\.vscode\analysis-practice\KAIST_25fall\advanced financial quant


In [28]:
from pathlib import Path 
import pandas as pd 
import sys 


BASE = Path("./NeoMatrixAI")
print(BASE.resolve())

C:\Users\User\.vscode\analysis-practice\KAIST_25fall\advanced financial quant\NeoMatrixAI


In [29]:
# strategy_verify (strategy 파일이 제대로 작성되었는지 확인)

In [30]:
sys.path += [
    str(BASE),
    str(BASE / "futures"),
    str(BASE / "module"),
    str(BASE / "module" / "futures"),
]

# loading_sample_data 
df = pd.read_csv(BASE / "df_sample.csv", parse_dates=['datetime']).set_index('datetime').sort_index()
print("shape:", df.shape)

# strategy_import 
# from futures.simple_momentum import simple_momentum as strategy
# from futures.simple_momentum import simple_momentum_config as config
# strategy_name = "simple_momentum"          

from futures.mean_reversion import mean_reversion as strategy
from futures.mean_reversion import mean_reversion_config as config
strategy_name = "mean_reversion"
   
# verify_strategy 
config_dict = {"strategy_config": config.strategy_config}
weights = strategy.strategy(df, config_dict)
print("weights:", weights)
print("total_weights =", sum(abs(v) for v in weights.values()))


shape: (4320, 10)
weights: {}
total_weights = 0


In [31]:
# backtesting 

USER_KEY = "6189628ae35ec9972d406ac67a09bf6c"      #  Input your User key
DATA_KEY = "a71eaf04-802f-40be-93c2-5bee2548f4db"  #  Fixed Data Api key

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")
print("DATA_KEY:", DATA_KEY if DATA_KEY else "Not found")

USER_KEY: 6189628ae35ec9972d406ac67a09bf6c
DATA_KEY: a71eaf04-802f-40be-93c2-5bee2548f4db


In [32]:
import requests

root_url = f'https://zipline.fin.cloud.ainode.ai/'
headers={"API-KEY": USER_KEY}
requests.get(root_url, headers=headers).json()

{'message': 'API Server is Running.', 'status': 200, 'User status': 'active'}

In [33]:
file_path = (BASE / "futures" / strategy_name / f"{strategy_name}.py").resolve()

if not file_path.exists():
    raise FileNotFoundError(f"file_not_found: {file_path}")

end_point = "upload/strategy"
url = f"{root_url.rstrip('/')}/{end_point}"

params = {
    "tradeType": "futures",
    "overwrite": True,  
}

with open(file_path, "rb") as f:
    files = {"file": (file_path.name, f, "text/x-python")}
    resp = requests.post(url, headers=headers, params=params, files=files, timeout=60)

print("HTTP:", resp.status_code)
print("RAW :", resp.text[:500])  
resp.raise_for_status()          
data = resp.json()
print("Response:", data)

HTTP: 200
RAW : {"message":"'mean_reversion' Strategy Updated Successfully!"}
Response: {'message': "'mean_reversion' Strategy Updated Successfully!"}


In [34]:
# Strategy configs
strategy_config_params = {
  "rebalancing_config": {             # Rebalancing settings
    "rebalancing_interval_hours": 72, ## Rebalancing cycle (choose between 6, 12, 24, and 72 hours)
  },
  "strategy_config": {
    "lookback": 60,              # Lookback period for momentum calculation
    "long_allocation_pct": 0.6,  # Proportion of capital allocated to longs
    "short_allocation_pct": 0.4  # Proportion of capital allocated to shorts
    }
}

In [35]:
# System configs
start_date_str = "2025-08-01"
end_date_str = "2025-08-30"
lookback_min = 60 # Max lookback minutes the script needs for data history
initial_capital = 10000000
leverage = 100
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BCHUSDT', 'LTCUSDT',
           'ADAUSDT', 'ETCUSDT', 'TRXUSDT', 'DOTUSDT', 'DOGEUSDT']

generate_report_flag = True

In [36]:
# run backtest 

In [37]:
import json
from datetime import datetime

path = BASE / "backtest_report"

request_payload = {
    "data_apikey": DATA_KEY,
    "strategy": strategy_name,
    "strategy_config": strategy_config_params,
    "start_date": start_date_str,
    "end_date": end_date_str,
    "lookback_minutes": lookback_min,
    "capital": initial_capital,
    "leverage": leverage,
    "symbols": symbols,
    "calendar": "24/7",
    "frequency": "minute",
    "generate_pyfolio_report": generate_report_flag
}

endpoint = '/run/futures/backtest'
url = root_url + endpoint

try:
    response = requests.post(url, json=request_payload, headers=headers)
    response.raise_for_status()
    print(f"\n--- Backtest execution successful (Status Code: {response.status_code}) ---")

    try:
        result_data = response.json()
        report_type = result_data.get('report_type') # Check report type

        if report_type == 'html':
            print("Report Type: HTML Report included.")
            html_content = result_data.get('html_content')
            logs = result_data.get('logs', 'No stderr logs received.')
            stdout_logs = result_data.get('stdout', 'No stdout received.')

            if html_content:
                if not os.path.exists(path):
                    os.makedirs(path)

                report_filename = os.path.join(path, f"{datetime.now().strftime('%Y-%m-%d %H-%M-%S')}_{strategy_name}_backtest_report.html")
                try:
                    with open(report_filename, "w", encoding="utf-8") as f:
                        f.write(html_content)
                    print(f"HTML report received and saved successfully as '{report_filename}'.")
                except Exception as e:
                    print(f"ERROR: Failed to save received HTML report: {e}")
                    print("\n--- Received HTML Content (Snippet) ---")
                    print(html_content[:1000] + "...") # Output some content when saving fails
            else:
                print("WARN: Report type was 'html' but no HTML content found in response.")

            # print logging
            print("\n--- Execution Logs (stderr) ---")
            print(logs)
            if stdout_logs:
                print("\n--- Execution Output (stdout) ---")
                print(stdout_logs)

        elif report_type == 'logs_only':
            print(f"Report Type: Logs Only (Report generation skipped).")
            print(f"Message: {result_data.get('message')}")
            print("\n--- Execution Logs (stderr) ---")
            print(result_data.get('logs', 'No stderr logs received.'))
            if 'stdout' in result_data:
                 print("\n--- Execution Output (stdout) ---")
                 print(result_data.get('stdout', 'No stdout received.'))
        else:
            print(f"WARN: Received successful response with unknown report_type: '{report_type}'")
            print("\n--- Full JSON Response ---")
            print(json.dumps(result_data, indent=2, ensure_ascii=False))

    except json.JSONDecodeError:
        print("ERROR: Failed to decode JSON response from successful API call.")
        print(f"Content-Type: {response.headers.get('content-type', 'N/A')}")
        print("\n--- Received Raw Content (First 1000 chars) ---")
        print(response.text[:1000] + "...")

except requests.exceptions.Timeout:
    print(f"\n--- API call failed: Timeout) ---")
except requests.exceptions.HTTPError as e:
    print(f"\n--- API call failed: HTTP Error {e.response.status_code} ---")
    try:
        error_details = e.response.json()
        print("Error details:")
        print(json.dumps(error_details, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("Error response content (Non-JSON):")
        print(e.response.text)
except requests.exceptions.RequestException as e:
    print(f"\n--- API call failure: Request Error ---")
    print(f"Error connecting to or requesting the API server ({url}): {e}")
except Exception as e:
    print(f"\n--- Unexpected error occurred ---")
    print(f"Error type: {type(e).__name__}, Content: {e}")


--- Backtest execution successful (Status Code: 200) ---
Report Type: HTML Report included.
HTML report received and saved successfully as 'NeoMatrixAI\backtest_report\2025-09-16 00-59-21_mean_reversion_backtest_report.html'.

--- Execution Logs (stderr) ---
[2025-09-15 15:58:27+0000] Backtest script execution started.
INFO: Command line arguments parsed.

--- Runtime Configuration ---
  - user_key: Provided (Masked)
  - data_apikey: Provided (Masked)
  - strategy_config_json: Provided (Number of parameters: 20)
  - start_date: 2025-08-01
  - end_date: 2025-08-30
  - lookback_minutes: 60
  - capital: 10000000.0
  - leverage: 100.0
  - symbols: ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BCHUSDT', 'LTCUSDT', 'ADAUSDT', 'ETCUSDT', 'TRXUSDT', 'DOTUSDT', 'DOGEUSDT']
  - calendar: 24/7
  - frequency: minute
  - output_file: /app/backtest/temp_backtest_results/pyfolio_report_2a16da58-d0a8-4cbf-86e3-7c4f556e621c.html
  - generate_report: True
-----------------------------
INFO: Loading dynamic strate

# trading 

In [51]:
strategy_name = "simple_momentum"
# Upload or Update strategy file
file_path = (BASE / "futures" / strategy_name / f"{strategy_name}.py").resolve()

if not file_path.exists():
    raise FileNotFoundError(f"file_not_found: {file_path}")

end_point = "upload/strategy"
url = f"{root_url.rstrip('/')}/{end_point}"

params = {
    "tradeType": "futures",
    "overwrite": True,  
}

with open(file_path, "rb") as f:
    files = {"file": (file_path.name, f, "text/x-python")}
    resp = requests.post(url, headers=headers, params=params, files=files, timeout=60)

print("HTTP:", resp.status_code)
print("RAW :", resp.text[:500])  
resp.raise_for_status()          
data = resp.json()
print("Response:", data)

HTTP: 200
RAW : {"message":"'simple_momentum' Strategy Updated Successfully!"}
Response: {'message': "'simple_momentum' Strategy Updated Successfully!"}


In [52]:
# Check strategy file upload
endpoint = 'upload/check/strategy'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy.py
import pandas as pd
import numpy as np

def strategy(df, config_dict):
    # --- Input validation ---
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")
    if df.empty:
        raise ValueError("Input DataFrame is empty.")
    if not isinstance(config_dict, dict):
        raise TypeError("config_dict must be a dictionary.")

    # --- Load strategy-specific config ---
    strategy_specific_config = config_dict.get("strategy_config", {})
    lookback = strategy_specific_config.get("lookback", 20)
    long_allocation_pct = strategy_specific_config.get("long_allocation_pct", 0.7)
    short_allocation_pct = strategy_specific_config.get("short_allocation_pct", 0.3)

    if long_allocation_pct + short_allocation_pct > 1.0:
        raise ValueError("Sum of long and short allocation percentages cannot exceed 1.0")

    # --- Momentum calculation ---
    momentum_scores = {}
    for symbol in df.columns:
        series = df[symbo

In [59]:
strategy_config_name = strategy_name + "_config"
file_path = (BASE / "futures" / strategy_name / f"{strategy_config_name}.py").resolve()

if not file_path.exists():
    raise FileNotFoundError(f"file_not_found: {file_path}")

endpoint = 'upload/strategy-config'
url = root_url + end_point

params = {
    "tradeType": "futures",
    "overwrite": True,  # True면 기존 config 덮어쓰기
}

headers = {"API-KEY": USER_KEY}

with open(file_path, "rb") as f:
    files = {"file": (file_path.name, f, "text/x-python")}
    resp = requests.post(url, headers=headers, params=params, files=files, timeout=60)

print(resp.json())

{'message': "'simple_momentum_config' Strategy Updated Successfully!"}


In [ ]:
# run-system
endpoint = 'command/run-system'
url = root_url + endpoint

data = {
    "tradeType": "futures",
    "strategy_name": strategy_name,
    "trade_mode": "rebalancing",
    "trade_env": "live"
}

response = requests.post(url, headers=headers, json=data)

try:
    session_id = response.json()['session_id']
    print('session_id :',response.json()['session_id'])
    print('dashboard :',response.json()['dashboard_url'])
except:
    print('Error :',response.json()['message']['message'])

    session_id = response.json()['message']['session_id']
    print('session_id id :',session_id)

    print('dashboard :',response.json()['message']['dashboard_url'])

In [ ]:
# Terminate process
# You must execute terminate to force liquidation of the current process.
endpoint = 'command/terminate'
url = root_url + endpoint

params = {"session_id": f"{session_id}", "tradeType": 'futures'}

response = requests.get(url, headers=headers, params=params)
print(response.json())

In [12]:
from module.futures import position
from module.futures import trade

df_all_position = position.all_positions(USER_KEY, 'usdt-futures', 'USDT')
df_all_position 


""
